In [ ]:
import hashlib

def shake256_bits(seed: str, bits: int) -> str:
    """Generates a SHAKE-256 pseudo-random bitstream of given length from a string seed."""
    shake = hashlib.shake_256()
    shake.update(seed.encode())
    num_bytes = (bits + 7) // 8
    byte_stream = shake.digest(num_bytes)
    return ''.join(f"{byte:08b}" for byte in byte_stream)[:bits]

def embed_tms_into_stream(TMS: str, template1: str, template2: str, seed="default") -> str:
    """
    Embeds a 16-bit TMS value into a 256-bit SHAKE-256 stream using two templates.

    Parameters:
        TMS (str): 16-bit binary string
        template1 (str): Template for TMS_lower (first 128 bits)
        template2 (str): Template for TMS_upper (second 128 bits)
        seed (str): Input seed for SHAKE-256

    Returns:
        Modified 256-bit binary string with embedded TMS.
    """
    assert len(TMS) == 16, "TMS must be a 16-bit binary string."

    stream = shake256_bits(seed, 256)
    stream = list(stream)  # Convert to list for mutability

    TMS_lower = TMS[8:]  # Last 8 bits
    TMS_upper = TMS[:8]  # First 8 bits

    # Embed TMS_lower using template1 in first 128 bits
    p1 = ''.join(stream[:128]).find(template1)
    if p1 != -1:
        insert_pos = p1 + len(template1)
        for i in range(8):
            stream[insert_pos + i] = TMS_lower[i]
    else:
        raise ValueError("Template1 not found in first 128 bits.")

    # Embed TMS_upper using template2 in second 128 bits
    p2_offset = 128
    p2 = ''.join(stream[128:]).find(template2)
    if p2 != -1:
        insert_pos = p2_offset + p2 + len(template2)
        for i in range(8):
            stream[insert_pos + i] = TMS_upper[i]
    else:
        raise ValueError("Template2 not found in second 128 bits.")

    return ''.join(stream)



In [ ]:
# === Example usage ===
if __name__ == "__main__":
    # Example TMS: 0xA5F3 = '1010010111110011'
    TMS1 = '1010010111110011'
    TMS2 = '0011110000010111'
    # Example templates
    template1 = '0010'  # For TMS_lower (F3)
    template2 = '1100'  # For TMS_upper (A5)

    encrypted_stream1 = embed_tms_into_stream(TMS1, template1, template2, seed="test_seed")
    encrypted_stream2 = embed_tms_into_stream(TMS2, template1, template2, seed="test_seed")

    print(f"Encrypted Bitstream 1:\n{encrypted_stream1}")
    print(f"Length: {len(encrypted_stream1)} bits")


    print(f"Encrypted Bitstream 2:\n{encrypted_stream2}")
    print(f"Length: {len(encrypted_stream2)} bits")
